In [2]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import re

In [2]:
#
#* buy a place
df = pd.DataFrame()
# r = requests.get("https://sh.lianjia.com/ershoufang/pg3rs%E9%BB%84%E6%B5%A6%E5%8C%BA/")

for i in range(2, 29):
    link = f"https://sh.lianjia.com/ershoufang/pg{i}rs%E9%BB%84%E6%B5%A6%E5%8C%BA/"
    print(link)
    tm = pd.DataFrame()
    r = requests.get(link)
    t = BeautifulSoup(r.content.decode()).find_all('ul', class_='sellListContent')[0]
    t = t.find_all('div', class_="info clear")
    for i in range(len(t)):
        tm.loc[i, 'address'] = t[i].find(attrs={'data-el': 'region'}).text
        tm.loc[i, 'info']  = t[i].find('div', class_ = 'address').text
        tm.loc[i, 'price'] = t[i].find('div', class_ = 'priceInfo').text
    df = pd.concat([df, tm])
    # time.sleep(5)
    # break
buy_df = df.copy(deep=True)
buy_df= pd.concat([buy_df, pd.DataFrame(buy_df['info'].str.split('|'  ,expand=True)).iloc[0:, 0:7]], axis = 1)
buy_df.columns =  ['address', 'info', 'price' ,'rtype', 'sqr', 'dir', 'dec', 'heights', 'built', 'btype']
buy_df[['price', 'price_per_sqr']] = buy_df['price'].str.replace('元/平', '').str.replace(',', '') .str.split('万', expand=True)
buy_df['price'] = buy_df['price'].astype(float)*10000
buy_df# .to_csv('./dataup/huangpu_fangjia.csv', index = None, encoding='utf_8_sig')

# 

In [5]:
buy_df = pd.read_csv('./dataup/huangpu_fangjia.csv',)
buy_df

,address,info,price,rtype,sqr,dir,dec,heights,built,btype,price_per_sqr
0,南园小区,2室1厅 | 72.97平米 | 南 西南 | 简装 | 中楼层(共29层) | 1994年...,5500000.0,2室1厅,72.97平米,南 西南,简装,中楼层(共29层),1994年建,塔楼,75374
1,蒙自新村,2室1厅 | 56.82平米 | 东南 | 精装 | 中楼层(共7层) | 1988年建 | 板楼,5350000.0,2室1厅,56.82平米,东南,精装,中楼层(共7层),1988年建,板楼,94157
2,局后小区,1室0厅 | 35.48平米 | 南 | 精装 | 低楼层(共6层) | 1982年建 | 板楼,3500000.0,1室0厅,35.48平米,南,精装,低楼层(共6层),1982年建,板楼,98648
3,新华南园大厦,2室1厅 | 85.62平米 | 南 | 简装 | 高楼层(共31层) | 2004年建 | 塔楼,7100000.0,2室1厅,85.62平米,南,简装,高楼层(共31层),2004年建,塔楼,82925
4,南园新村,1室1厅 | 34.31平米 | 东南 | 简装 | 高楼层(共6层) | 1988年建 | 板楼,3550000.0,1室1厅,34.31平米,东南,简装,高楼层(共6层),1988年建,板楼,103469
...,...,...,...,...,...,...,...,...,...,...,...
805,越纪公寓,2室1厅 | 85.87平米 | 南 | 简装 | 中楼层(共25层) | 1992年建 |...,9900000.0,2室1厅,85.87平米,南,简装,中楼层(共25层),1992年建,板塔结合,115291
806,瞿四小区,2室0厅 | 47.4平米 | 南 | 简装 | 低楼层(共6层) | 1989年建 | 板楼,4680000.0,2室0厅,47.4平米,南,简装,低楼层(共6层),1989年建,板楼,98735
807,豫景公寓,1室1厅 | 69.07平米 | 南 | 简装 | 中楼层(共26层) | 2000年建 | 塔楼,8590000.0,1室1厅,69.07平米,南,简装,中楼层(共26层),2000年建,塔楼,124367
808,银南大厦,2室1厅 | 79.99平米 | 西南 | 精装 | 高楼层(共22层) | 1991年建 ...,7880000.0,2室1厅,79.99平米,西南,精装,高楼层(共22层),1991年建,塔楼,98513


In [168]:
##
#* rent a plkace
df = pd.DataFrame()
for g in range(2,55):
    link = f"https://sh.lianjia.com/zufang/pg{g}rs%E9%BB%84%E6%B5%A6/"
    tm = pd.DataFrame()
    r = requests.get(link)
    try:
        for i in BeautifulSoup(r.content.decode()).find_all('div', class_='content__list--item'):
            dtl = i.find_all('div')[0]
            tm.loc[g, 'address'] = dtl.find('a', class_ = 'twoline').text.strip()
            # Use regular expression to extract the area
            area_pattern = re.compile(r'(\d+\.\d+)㎡')
            area_element = dtl.find(text=area_pattern)
            
            tm.loc[g, 'sqr'] = area_element.strip().replace('㎡', '')
            price_span = dtl.find('span', class_='content__list--item-price')

            # Extract the price value
            if price_span:
                price_em = price_span.find('em')
                if price_em:
                    tm.loc[g, 'price'] = price_em.text
                else:
                    print("Price value not found")
            else:
                print("Price span not found")
            g += 1
        df = pd.concat([df, tm])
    except:
        print(link)

        pass
    time.sleep(4)
df[['address', 'type', 'dir']] = df['address'].str.split(' ', expand=True).iloc[:, 0:3]
df[['renttype', 'address']] = df['address'].str.split('·', expand=True)
df# .to_csv('./dataup/huangpu_fangzu.csv', index = None, encoding='utf_8_sig')

    

https://sh.lianjia.com/zufang/pg7rs%E9%BB%84%E6%B5%A6/
https://sh.lianjia.com/zufang/pg10rs%E9%BB%84%E6%B5%A6/


In [7]:
df = pd.read_csv('./dataup/huangpu_fangzu.csv',)

In [12]:
df[df.address.str.contains('南园')].sort_values('sqr').tail(9)

,address,sqr,price,type,dir,renttype
119,南园新村,53.04,7200.0,2室1厅,南,整租
57,南园小区,64.52,7690.0,2室1厅,东,整租
619,南园小区,66.10,8000.0,2室1厅,南,整租
645,南园小区,66.10,8000.0,2室1厅,南,整租
825,南园小区,66.14,8000.0,2室1厅,南,整租
404,南园小区,66.25,8300.0,2室1厅,东南,整租
71,南园小区,66.50,8590.0,2室1厅,南,整租
590,新华南园大厦,68.20,8500.0,2室1厅,南,整租
73,新华南园大厦,75.46,6490.0,1室1厅,南,整租


In [11]:
buy_df[buy_df['address'].str.contains('南园')].sort_values('sqr').drop('info', axis = 1)

,address,price,rtype,sqr,dir,dec,heights,built,btype,price_per_sqr
4,南园新村,3550000.0,1室1厅,34.31平米,东南,简装,高楼层(共6层),1988年建,板楼,103469
216,南园新村,4350000.0,2室0厅,44.06平米,南,精装,低楼层(共6层),1988年建,板楼,98730
786,新华南园大厦,4300000.0,1室1厅,47.7平米,东北,简装,低楼层(共31层),2004年建,塔楼,90147
359,南园新村,5500000.0,2室0厅,52.25平米,南,精装,高楼层(共6层),1988年建,板楼,105264
307,新华南园大厦,5250000.0,1室1厅,54.59平米,东南,简装,中楼层(共31层),2004年建,塔楼,96172
222,南园小区,4150000.0,1室1厅,57.16平米,西南,毛坯,低楼层(共29层),1994年建,塔楼,72604
747,南园小区,6500000.0,2室1厅,64.52平米,南 东南,简装,中楼层(共29层),1994年建,塔楼,100744
9,南园小区,6590000.0,2室1厅,66.25平米,西南,简装,高楼层(共29层),1992年建,塔楼,99472
125,新华南园大厦,6680000.0,2室1厅,68.2平米,南 北,精装,低楼层(共28层),2004年建,塔楼,97948
640,南园小区,5300000.0,2室1厅,69.91平米,西 西北,简装,低楼层(共29层),1994年建,塔楼,75812


ms随便选一套
-- 
<br>
南园小区	64.52	7690.0	2室1厅	东	整租

--

南园小区	6500000.0	2室1厅	64.52平米	南 东南	简装	中楼层(共29层)	1994年建	塔楼	100744

rent

In [13]:
# the flat with 64 sqr metr, 2b1b
# rent : 2% increase as inflation per year for 20 years
ttrent = 0
rent_ = 7690.0
for i in range(20):
    ttrent += rent_ * 12
    rent_ = rent_*1.02 # 
    print(f"{i+1} year ttl rent: " ,str (ttrent))
    i+=1

1 year ttl rent:  92280.0
2 year ttl rent:  186405.6
3 year ttl rent:  282413.712
4 year ttl rent:  380341.98624
5 year ttl rent:  480228.8259648
6 year ttl rent:  582113.402484096
7 year ttl rent:  686035.6705337779
8 year ttl rent:  792036.3839444534
9 year ttl rent:  900157.1116233425
10 year ttl rent:  1010440.2538558093
11 year ttl rent:  1122929.0589329256
12 year ttl rent:  1237667.6401115842
13 year ttl rent:  1354700.992913816
14 year ttl rent:  1474075.0127720921
15 year ttl rent:  1595836.513027534
16 year ttl rent:  1720033.2432880846
17 year ttl rent:  1846713.9081538464
18 year ttl rent:  1975928.1863169235
19 year ttl rent:  2107726.750043262
20 year ttl rent:  2242161.285044127


---
buy  = 6500000 # 6.5 million

 interest rate 4.55%

 prepay 35%

In [15]:
# Loan details
#
call_ = 6500000
for i in [.35,.5,.7,.9]: # 首付比例
    init_pay_rate = i
    annual_interest_rate = .031
    # annual_interest_rate = 0.0455  # Annual interest rate (4.55%)

    principal = call_ - init_pay_rate*call_  # Loan amount
    loan_term_years = 10  # Loan term in years

    # Calculate monthly interest rate and total number of payments
    monthly_interest_rate = annual_interest_rate / 12
    total_payments = loan_term_years * 12

    # Calculate the annual payment amount
    annual_payment = principal * (monthly_interest_rate * (1 + monthly_interest_rate) ** total_payments) / ((1 + monthly_interest_rate) ** total_payments - 1)

    # Calculate the total amount paid over the life of the loan
    total_amount_paid = annual_payment * total_payments + init_pay_rate*call_

    # Print the results
    print(f"""
    when init pay per is : {str(init_pay_rate)} \t\t
    interest rate is: {str(annual_interest_rate)} 
    Total Amount Paid: {total_amount_paid:.2f}
    ---
    """)



    when init pay per is : 0.35 		
    interest rate is: 0.031 
    Total Amount Paid: 7194067.66
    ---
    

    when init pay per is : 0.5 		
    interest rate is: 0.031 
    Total Amount Paid: 7033898.20
    ---
    

    when init pay per is : 0.7 		
    interest rate is: 0.031 
    Total Amount Paid: 6820338.92
    ---
    

    when init pay per is : 0.9 		
    interest rate is: 0.031 
    Total Amount Paid: 6606779.64
    ---
    


$$PMT = \frac{P \cdot r \cdot (1 + r)^n}{(1 + r)^n - 1}$$


In [274]:
call_ * 1.02 ** 20

9658658.073859306